In [1]:
import pandas as pd
import numpy as np
np.random.seed(456)
import re
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import os
from sklearn.model_selection import train_test_split

In [2]:
data=pd.read_csv('/content/drive/MyDrive/애쓰는 감자/data/new_data/train_data_final.csv',dtype={'KEDCD':str,'BZ_PPOS_ITM_CTT':str,'KSIC10_BZC_CD':str,'mid':str,'Big':str,'small':str})

data.head()

,KEDCD,BZ_PPOS_ITM_CTT,KSIC10_BZC_CD,mid,Big,small
0,7695516.0,농산물 판매업 전자상거래업 통신판매업농임산물 건강기능식품 건강보조식품 일반식품...,G46209,G46,G,462
1,8177802.0,건축공사업 토목공사업 토공사업 토목건축공사업 철근콘크리트공사업 실내인테리...,F41229,F41,F,412
2,8233352.0,음식점업 경양식 퓨전 한식 중식 일식 식품 제조 가공 유통 판매 도소매업 제과...,I56111,I56,I,561
3,991833.0,변호사의 직무에 속하는 업무 공증인의 직무에 속하는 업무 변호사법 제조에 속하...,M71101,M71,M,711
4,7305137.0,일반폐기물처리업 재활용사업 정화조 청소업 위 각항에 부대되는 사업일체,E38210,E38,E,382


In [3]:
data['small'].unique()

array(['462', '412', '561', '711', '382', '763', '463', '467', '381',
       '461', '465', '713', '181', '969', '162', '464', '281', '591',
       '339', '701', '424', '715', '661', '941', '681', '292', '529',
       '855', '949', '551', '421', '751', '911', '477', '468', '471',
       '682', '451', '012', '474', '493', '479', '107', '011', '265',
       '411', '582', '311', '649', '620', '856', '759', '353', '020',
       '222', '752', '031', '383', '351', '014', '662', '466', '334',
       '331', '423', '062', '205', '262', '204', '742', '422', '241',
       '212', '291', '641', '452', '729', '231', '732', '101', '251',
       '141', '259', '303', '872', '143', '425', '473', '581', '857',
       '213', '651', '132', '639', '071', '761', '201', '592', '741',
       '478', '721', '472', '192', '172', '521', '320', '271', '106',
       '631', '239', '912', '284', '161', '492', '263', '475', '302',
       '901', '261', '103', '714', '282', '104', '272', '952', '642',
       '612', '202',

In [4]:
data[data['small'].isnull()]

,KEDCD,BZ_PPOS_ITM_CTT,KSIC10_BZC_CD,mid,Big,small


In [5]:
len(data)

1123776

In [6]:
data=data.dropna(subset=['small'])

In [7]:
len(data)

1123776

In [8]:
x_train=data[['KEDCD','BZ_PPOS_ITM_CTT']]
y_train=data['small']

In [9]:
x_train.shape, y_train.shape

((1123776, 2), (1123776,))

In [10]:
x_train.head()

,KEDCD,BZ_PPOS_ITM_CTT
0,7695516.0,농산물 판매업 전자상거래업 통신판매업농임산물 건강기능식품 건강보조식품 일반식품...
1,8177802.0,건축공사업 토목공사업 토공사업 토목건축공사업 철근콘크리트공사업 실내인테리...
2,8233352.0,음식점업 경양식 퓨전 한식 중식 일식 식품 제조 가공 유통 판매 도소매업 제과...
3,991833.0,변호사의 직무에 속하는 업무 공증인의 직무에 속하는 업무 변호사법 제조에 속하...
4,7305137.0,일반폐기물처리업 재활용사업 정화조 청소업 위 각항에 부대되는 사업일체


In [11]:
y_train.head()

0    462
1    412
2    561
3    711
4    382
Name: small, dtype: object

In [12]:
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

+ pip install konlpy
+ curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh
+ bash -x
+ mecab_dicdir=/usr/local/lib/mecab/dic/mecab-ko-dic
+ set -e
++ uname
+ os=Linux
+ [[ ! Linux == \L\i\n\u\x ]]
+ hash sudo
+ sudo=sudo
+ python=python3
+ hash pyenv
+ at_user_site=
++ check_python_site_location_is_writable
++ python3 -
+ [[ 1 == \0 ]]
+ hash automake
+ hash mecab
+ echo 'mecab-ko is already installed'
mecab-ko is already installed
+ [[ -d /usr/local/lib/mecab/dic/mecab-ko-dic ]]
+ echo 'mecab-ko-dic is already installed'
mecab-ko-dic is already installed
++ python3 -c 'import pkgutil; print(1 if pkgutil.find_loader("MeCab") else 0)'
+ [[ 1 == \1 ]]
+ echo 'mecab-python is already installed'
mecab-python is already installed
+ echo Done.
Done.


In [13]:
from konlpy.tag import Mecab
mecab = Mecab()

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,  ReduceLROnPlateau
from tensorflow.keras.regularizers import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam, RMSprop
import tensorflow as tf

In [15]:
stop_words = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을']

In [16]:
X_train = []

text = list(x_train['BZ_PPOS_ITM_CTT'])

for i in tqdm(range(len(text))):
  temp_X = []
  temp_X = mecab.nouns(text[i]) # 토큰화
  temp_X = [word for word in temp_X if not word in stop_words] # 불용어 제거
  temp_X = [word for word in temp_X if len(word) > 1]
  X_train.append(temp_X)

100%|██████████| 1123776/1123776 [07:16<00:00, 2572.22it/s]


In [17]:
X_train

[['농산물',
  '판매업',
  '전자',
  '상거래',
  '통신',
  '판매업',
  '임산물',
  '건강',
  '기능',
  '식품',
  '건강',
  '보조',
  '식품',
  '일반',
  '식품',
  '부동산',
  '매매',
  '임대',
  '대업',
  '부동산',
  '개발',
  '임산물',
  '판매업',
  '건강',
  '기능',
  '식품',
  '제조',
  '수입',
  '판매업',
  '건강',
  '보조',
  '식품',
  '제조',
  '수입',
  '판매업',
  '식품',
  '제조',
  '수입',
  '판매업',
  '교육',
  '교육',
  '강연',
  '세미나',
  '출판',
  '인쇄업',
  '인재',
  '파견',
  '호에',
  '관련',
  '부대사업',
  '일체'],
 ['건축',
  '사업',
  '목공',
  '사업',
  '토공',
  '사업',
  '토목건축',
  '사업',
  '철근',
  '콘크리트',
  '사업',
  '실내',
  '인테리어',
  '사업',
  '실내',
  '건축',
  '사업',
  '경공',
  '사업',
  '조경',
  '식재',
  '사업',
  '설계',
  '감리',
  '측량',
  '건축',
  '자재',
  '판매업',
  '임대업',
  '실내',
  '인테리어',
  '자재',
  '판매업',
  '각호',
  '관련',
  '소매업',
  '각호',
  '관련',
  '수출입',
  '무역업',
  '부동산',
  '임대',
  '매매',
  '각호',
  '관련',
  '부대사업',
  '일체'],
 ['음식점',
  '경양식',
  '퓨전',
  '한식',
  '중식',
  '일식',
  '식품',
  '제조',
  '가공',
  '유통',
  '판매',
  '소매업',
  '제과',
  '제빵',
  '제조',
  '판매',
  '소매업',
  '프랜차이즈',
  '각항',
  '관련',
  '컨설팅',
  '문업

In [18]:
import pickle
import gzip
with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/new_data/X_train_new.pickle', 'wb') as f:
    pickle.dump(X_train, f)

In [21]:

with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/new_data/Y_train_new.pickle', 'wb') as f:
    pickle.dump(y_train, f)

----- 실행 x------

In [2]:
import pickle
import gzip

with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/new_data/X_train_new.pickle','rb') as f:
  X_train = pickle.load(f)

In [ ]:
X_train_new

In [4]:
import pickle
import gzip
with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/new_data/Y_train_new.pickle','rb') as f:
  y_train = pickle.load(f)

In [ ]:
Y_train_new

0          462.0
1          412.0
2          561.0
3          711.0
4          382.0
           ...  
1141486    970.0
1141487    981.0
1141488    982.0
1141489    990.0
1141490    990.0
Name: small, Length: 1141491, dtype: float64

In [ ]:
with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/nan_list/small_train_new_nan.pickle','rb') as f:
  drop_idx = pickle.load(f)

In [ ]:
len(X_train), len(y_train),len(drop_idx)

(1141489, 1141489, 17710)

-------------------------------------

In [ ]:
data

,KEDCD,BZ_PPOS_ITM_CTT,KSIC10_BZC_CD,mid,Big,small
0,7695516.0,농산물 판매업 전자상거래업 통신판매업농임산물 건강기능식품 건강보조식품 일반식품...,G46209,G46,G,462
1,8177802.0,건축공사업 토목공사업 토공사업 토목건축공사업 철근콘크리트공사업 실내인테리...,F41229,F41,F,412
2,8233352.0,음식점업 경양식 퓨전 한식 중식 일식 식품 제조 가공 유통 판매 도소매업 제과...,I56111,I56,I,561
3,991833.0,변호사의 직무에 속하는 업무 공증인의 직무에 속하는 업무 변호사법 제조에 속하...,M71101,M71,M,711
4,7305137.0,일반폐기물처리업 재활용사업 정화조 청소업 위 각항에 부대되는 사업일체,E38210,E38,E,382
...,...,...,...,...,...,...
1141486,NaN,가구 내 고용활동,NaN,NaN,NaN,970
1141487,NaN,자가 소비를 위한 가사 생산 활동,NaN,NaN,NaN,981
1141488,NaN,자가 소비를 위한 가사 서비스 활동,NaN,NaN,NaN,982
1141489,NaN,주한 외국 공관,NaN,NaN,NaN,990


In [ ]:
drop_idx = []
for i,ksic in enumerate(data['small']):  # 소분류 없는 인덱스 찾기
  if len(str(ksic))<3:
    drop_idx.append(i)

In [ ]:
len(drop_idx)

17710

In [ ]:
with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/nan_list/small_train_new_nan.pickle', 'wb') as f:
    pickle.dump(drop_idx, f)

In [ ]:
data.reset_index(drop=True,inplace=True)

In [ ]:
data.drop(drop_idx,axis=0,inplace=True)

In [ ]:
len(data['small'].unique())

233

In [ ]:
X_train = np.delete(X_train, drop_idx, axis=0).tolist()
#y_train = np.delete(y_train, drop_idx, axis=0).tolist()

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
len(y_train)

1141489

In [ ]:
X_train

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,  ReduceLROnPlateau
from tensorflow.keras.regularizers import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import *
import tensorflow.keras.backend as K
from datetime import datetime
from sklearn.model_selection import train_test_split

In [24]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [26]:
threshold = 11
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 53694
등장 빈도가 10번 이하인 희귀 단어의 수: 35117
단어 집합에서 희귀 단어의 비율: 65.4020933437628
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 0.21709119435848254


In [27]:
vocab_size = total_cnt - rare_cnt + 1 # 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거. 0번 패딩 토큰을 고려하여 +1
print('단어 집합의 크기 :',vocab_size)

단어 집합의 크기 : 18578


In [28]:
tokenizer = Tokenizer(num_words = vocab_size) # num_words = vocab_size
tokenizer.fit_on_texts(X_train)
token_X_train = tokenizer.texts_to_sequences(X_train)

In [29]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
y_train = ohe.fit_transform(y_train.values.reshape(-1,1)).toarray()

In [30]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [32]:
with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/new_data/token_x_train.pickle', 'wb') as f:
    pickle.dump(token_X_train, f)

In [6]:
with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/new_data/token_x_train.pickle','rb') as f:
  token_X_train= pickle.load(f)

In [ ]:
del X_train

In [ ]:
#y_train = np.delete(y_train, drop_idx, axis=0).tolist()

In [7]:
drop_train = [index for index, sentence in enumerate(token_X_train) if len(sentence) < 1]

In [8]:
#del_X_train = np.delete(token_X_train, drop_train, axis=0)
del_y_train = np.delete(y_train.values, drop_train, axis=0)
#print(len(del_X_train))
print(len(del_y_train))

1123771


In [9]:
del_y_train

array(['462', '412', '561', ..., '982', '990', '990'], dtype=object)

In [10]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
del_y_train = ohe.fit_transform(del_y_train.reshape(-1,1)).toarray()

In [11]:
del_y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [12]:
with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/new_data/del_y_train.pickle', 'wb') as f:
    pickle.dump(del_y_train, f)

#with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/new_data/del_X_train.pickle', 'wb') as f:
#    pickle.dump(del_X_train, f)

In [36]:
max_len = 300
pad_X_train = pad_sequences(del_X_train, maxlen = max_len)

In [37]:
pad_X_train

array([[    0,     0,     0, ...,     2,     9,     7],
       [    0,     0,     0, ...,     2,     9,     7],
       [    0,     0,     0, ...,     2,     9,     7],
       ...,
       [    0,     0,     0, ...,  2389,    68,   160],
       [    0,     0,     0, ...,  3937,   846, 12216],
       [    0,     0,     0, ...,   272,   846,   162]], dtype=int32)

In [38]:
with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/new_data/pad_X_train.pickle', 'wb') as f:
    pickle.dump(pad_X_train, f)